In [ ]:
# !wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl

In [ ]:
# !wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl

In [ ]:
# !wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl

In [ ]:
# !pip install simpletransformers

In [ ]:
# !pip install wandb

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import json
import torch
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline
from sklearn.metrics import *

from simpletransformers.classification import ClassificationModel

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''},  # remove spaces at the beginning
        {r'\([^)]*\)': ''}
    ]

    for rule in rules:
      for (k, v) in rule.items():
        regex = re.compile(k)
        text = regex.sub(v, text)

    text = text.rstrip()
    return text.lower()
  
def remove_URL( text):
        '''
        Function to remove url from text.
        @param text: (str) sentence
        @return: (str) clean text
        
        '''
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)
    
    
def remove_html( text):
        '''
        Function regex to clean text from html balises.
        @param text: (str) sentence 
        @return: (str) clean text 
        '''
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)
    
    

def remove_emoji( text):
        '''
        Function to remove emojis, symbols and pictograms etc from text
        @param text: (str) sentences 
        @return: (str) clean text 
        '''
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [ ]:
lisd = [1, 2, 3, 4, 5, 6, 7]
print(lisd[-2:]+lisd[:4])

[6, 7, 1, 2, 3, 4]


In [ ]:
def text_splitter(text, start=128, end=382):
    tokens = text.split()

    if len(tokens) > 510:
      new_text = ' '.join(tokens[:start] + tokens[-end:])
    else:
      new_text = text
    return new_text

In [ ]:
def get_X_y(data_json_file, is_test=False):
    count_ques_per_text = dict()

    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for i in range(len(json_list)):
            item = json.loads(json_list[i])
            text = item['passage']['text']

            if text.startswith('\"'):
              text = text[1:]

            if text.endswith('\"'):
              text = text[:len(text)]

            questions = item['passage']['questions']

            count_ques_per_text[i] = []
            for q in range(len(questions)):
                query = questions[q]['question']
                ans = questions[q]['answers']

                count_ques_per_text[i].append(len(ans))
                for a in ans:
                    X.append(text_splitter(text_cleaner(remove_emoji(remove_URL(remove_html(text.strip() + " " + query + " " + a['text']))))))
                    # X.append(text_cleaner(remove_emoji(remove_URL(remove_html(text.strip() + " " + a['text'])))))
                    if not is_test: y.append(a['label'])
                    
    if not is_test: return X, y, count_ques_per_text
    return X, count_ques_per_text

In [ ]:
X_train, y_train, count_ans_train = get_X_y('train.jsonl')
X_val, y_val, count_ans_val = get_X_y('val.jsonl')
X_test, count_ans_test = get_X_y('test.jsonl', is_test=True)

In [ ]:
X_train[0]

' но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали.  в парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть».  «имали они тут и девок...»  верховодил шпаной артемка-мыло, с вспененной белой головой.  людочка сколько ни пыталась усмирить лохмотья на буйной голове артемки, ничего у неё не получалось.  его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали.  да и не ради причёски приходил парень к людочке.  как только её руки становились занятыми ножницами и расчёской, артемка начинал хватать её за разные места.  людочка сначала увёртывалась от хватких рук артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажористого человека».  артемка заулюлюкал и со свистом стал л

In [ ]:
lens = []
for text in X_train:
  lens.append(len(text.split()))
print(max(lens))

lens = []
for text in X_val:
  lens.append(len(text.split()))
print(max(lens))

lens = []
for text in X_test:
  lens.append(len(text.split()))
print(max(lens))

510
510
435


In [ ]:
X_train += X_val
y_train += y_val

In [ ]:
train_df = pd.DataFrame({
    'text': X_train,
    'label': y_train
}, dtype='object')

train_df = train_df.sample(frac=1).reset_index(drop=True)

eval_df = pd.DataFrame({
    'text': X_val,
    'label': y_val
}, dtype='object')

eval_df = eval_df.sample(frac=1).reset_index(drop=True)

eval_df.head()

,text,label
0,стала она слепнуть. текст расплывается перед г...,0
1,спецподразделения воздушно-десантных войск пр...,0
2,"за первым фильмом ""голодные игры"" последуют ...",1
3,новый хозяин швейка — любитель животных и жен...,0
4,бой за титул чемпиона мира в супертяжелом вес...,0


In [ ]:
eval_df['label']

0       0
1       0
2       1
3       0
4       0
       ..
2230    0
2231    0
2232    0
2233    1
2234    0
Name: label, Length: 2235, dtype: object

In [ ]:
# train_df = train_df.astype({'label': torch.long})
# eval_df = eval_df.astype({'label': torch.long})

In [ ]:
train_df.dtypes

text     object
label    object
dtype: object

In [ ]:
# !rm val.jsonl

In [ ]:
# sweep_config = {
#     "method": "bayes",  # grid, random
#     "metric": {"name": "eval_loss", "goal": "minimize"},
#     "parameters": {
#         "random_seed": {"min": 42, 'max': 1024}
#     },
# }

In [ ]:
# import wandb

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="FinalBert")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 0daz195y
Sweep URL: https://wandb.ai/tishran/FinalBert/sweeps/0daz195y


In [ ]:
args = {'do_lower_case': True,
        'evaluate_during_training': True,
        'evaluate_during_training_verbose': True,
        'num_train_epochs': 3,
        'overwrite_output_dir': True,
        'learning_rate': 2e-5,
        'save_steps': 30000,
        'train_batch_size': 8,
        'eval_batch_size': 8,
        'max_seq_length': 512,
        'use_multiprocessing': False,
        'manual_seed': 935, #was 42
        'no_cache': True,
        'reprocess_input_data': True,
        "sliding_window": True}
model = ClassificationModel('bert', 'bert-base-multilingual-uncased', use_cuda=True, args=args) 
# def train():
#   wandb.init()

 

#   model.train_model(train_df, eval_df=eval_df)

#   model.eval_model(eval_df)

#   wandb.join()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
# wandb.agent(sweep_id, train)

In [ ]:
# import torch
# torch.cuda.empty_cache()

# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
model.train_model(train_df=train_df, eval_df=eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:391: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/14185 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1960 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:970: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Epoch 1 of 3:   0%|          | 0/1960 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
preds, raw_outputs = model.predict(X_val)
f1_score(y_val, preds, average='micro')

In [ ]:
from scipy.special import softmax

def get_probs(raw_outputs):
  probabilities = softmax(raw_outputs, axis=1)

  return probabilities

In [ ]:

def get_final(probabilities):
  final_reses = []
  for i in probabilities:
    if i[1] > 0.45:
      final_reses.append(1)
    else: final_reses.append(0)

  return final_reses

In [ ]:
def normalize_arr(arr):
  arr1 = arr.copy()
  for i in range(len(arr)):
    if len(arr[i]) > 1:
      arr1[i] = np.array([float((arr1[i][0][0] + arr1[i][1][0]) / 2), float((arr1[i][0][1] + arr1[i][1][1]) / 2)], dtype=np.float32)
    else:
      arr1[i] = np.array(arr1[i][0], dtype=np.float32)


  return np.array(arr1)

In [ ]:
def right_preds(probs, count):
  preds = []

  id_ans = 0
  for i in count.values():
    for j in i:
      added_ones = 0
      for k in range(j):
        if probs[id_ans][1] >= 0.45 and added_ones <= (j // 2) + 1:
          preds.append(1)
          added_ones += 1
        else:
          preds.append(0)
        
        id_ans += 1

  return preds

In [ ]:
raw_out = normalize_arr(raw_outputs)
probs_val = get_probs(raw_out)

In [ ]:
final_reses = right_preds(probs_val, count_ans_val)

In [ ]:
f1_score(y_val, final_reses, average='micro')

In [ ]:
# for i in range(len(preds)):
#     if preds[i] != y_val[i]:
#       print(X_val[i])


In [ ]:
def save_output(data, path, count_ans_test):

    with open(path, mode='w') as file:
      id_of_ques = 0
      id_of_ans = 0
      for i in range(len(count_ans_test)):
        res_line = dict()
        res_line["idx"] = i
        res_line["passage"] = dict()
        res_line["passage"]["questions"] = []

        for ques_idx in range(len(count_ans_test[i])):
          answers = dict()
          answers['idx'] = id_of_ques
          answers['answers'] = []
          for ans_idx in range(count_ans_test[i][ques_idx]):
            one_que = dict()
            one_que['idx'] = id_of_ans
            one_que['label'] = int(data[id_of_ans])
            answers['answers'].append(one_que)
            id_of_ans += 1

          res_line["passage"]["questions"].append(answers)
          id_of_ques += 1

        file.write(f"{json.dumps(res_line, ensure_ascii=False)}\n")

    print('Predictions saved!')

In [ ]:
preds_for_test, raw_test = model.predict(X_test)

In [ ]:
raw_t = normalize_arr(raw_test)
probs_test = get_probs(raw_t)
final_res_test = right_preds(probs_test, count_ans_test)

In [ ]:
save_output(final_res_test, '/content/drive/MyDrive/final.jsonl', count_ans_test)

In [ ]:
# !mv /content/outputs/best_model /content/drive/MyDrive/NLPforNTI/LastModel